# Run experiment salinity forecasting with LSTM


---


The aim of this script is to build a LSTM model able to forecast the salinity at the estuary mouth.

## Install & import library

In [ ]:
!pip install tensorflow
!pip install pandas
!pip install scikit-learn
!pip install keras-tuner --upgrade

In [ ]:
# Data Manipulation
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt

# TensorFlow / Keras
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Other
import warnings
import os
from math import sqrt

In [ ]:
warnings.filterwarnings('ignore')

## Function definition

### Function to load the data and extract sequence from timeseries

In [ ]:
def load_dataset(path_dataset):
  df = None
  if not os.path.isfile(path_dataset):
      print('Error! Invalid filename.')
  else:
      print(path_dataset + ' is a valid file.')
      df = pd.read_excel(path_dataset, index_col = 'Date')
  return df

In [ ]:
def df_to_X_y(df, features, output, n_features, window_size=1, step_ahead=0):
    X, y = [], []
    for i in range(len(df) - (window_size + step_ahead)):
        X.append(df.iloc[i:i + window_size][features].to_numpy())
        y.append(df.iloc[i + window_size: i + window_size + step_ahead][output].to_numpy())
    X = np.array(X)
    y = np.array(y)
    return np.reshape(X, (X.shape[0], window_size, n_features)), np.reshape(y, (y.shape[0], step_ahead))

### Function to visualize the model results and compute metrics

In [ ]:
def plot_perfect_fit(obs, pred, plot_title, xy_lim, add_bound=False, bound=0):
  xy_line = np.linspace(0, xy_lim, xy_lim)
  legend_str = ['Observations', 'Perfect prediction']

  plt.grid(visible=True)
  plt.scatter(obs, pred)
  plt.plot(xy_line, xy_line, 'k')

  if add_bound:
      plt.plot(xy_line, xy_line + bound*xy_line/100,'r--')
      plt.plot(xy_line, xy_line - bound*xy_line/100,'r--')
      legend_str = ['Observations', 'Perfect prediction', str(bound) +'% of deviation']

  plt.title(plot_title)
  plt.axis([0, xy_lim, 0, xy_lim])
  plt.legend(legend_str)
  plt.xlabel('True response (psu)')
  plt.ylabel('Predicted response (psu)')

In [ ]:
def plot_timeseries(obs, pred, plot_title, xy_lim):
  plt.grid(visible=True)
  plt.plot(obs)
  plt.plot(pred)
  plt.title(plot_title)
  plt.ylim([0, xy_lim])

  plt.legend(['Observations', 'LSTM predictions'])
  plt.xlabel('Time steps')
  plt.ylabel('Sul (psu)')

In [ ]:
def compute_metrics(obs, pred):
  rmse=sqrt(mean_squared_error(obs,pred))
  mae=mean_absolute_error(obs,pred)
  r2 = r2_score(obs, pred)
  print('RMSE: {}\nMAE: {}\nR2: {}'.format(rmse, mae, r2))

### Function to define the search space of hyperparameters for LSTM

In [ ]:
def build_model(hp):
  model = Sequential(name='LSTM_Neural_Network')

  model.add(LSTM(units=hp.Int(name='Units_LSTM_1', min_value=32, max_value=128, step=32),
                 input_shape=(window_size, n_features),
                 return_sequences=True))
  model.add(Dropout(hp.Float(name='dropout_factor_1', min_value=0.1, max_value=0.9, step=0.1)))
  model.add(LSTM(units=hp.Int(name='Units_LSTM_2', min_value=32, max_value=128, step=32)))
  model.add(Dropout(hp.Float(name='dropout_factor_2', min_value=0.1, max_value=0.9, step=0.1)))

  # Define the optimizer learning rate as a hyperparameter.
  lr = hp.Float(name='learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

  model.compile(
      optimizer=Adam(learning_rate=lr),
      loss=['mean_squared_error'],
      metrics=[RootMeanSquaredError()],
      )
  return model

## Run the experiment

### Define setting experiment
* *window_size*: the number of previous time steps should be considered into the sequence
* *n_features*: the number of features
* *step_ahead*: the number of steps ahead that the model should predict in sequence


In [ ]:
features = ['Qriver']
output = ['Sul']
window_size = 4
step_ahead = 7
n_features = len(features)
###################################
directory_network='network-trials\\'
project_name_network='lstm-qriver'
###################################
path_file = '..\models\LSTM\only-qriver\\'
path_filename_results = path_file+str(window_size)+'DayInp_OutDay'+ str(step_ahead)+'.xlsx'
path_filename_trained_network = path_file+str(window_size)+'DayInp_OutDay'+ str(step_ahead)+'.h5'

### Load the dataset

In [ ]:
df = load_dataset('..\data\processed\input-features-sul.xlsx')
print('------------------------------------------------------------------------')
print('df shape: {}'.format(df.shape))
print('\ndata types: \n{}'.format(df.dtypes))
df

In [ ]:
dataset = df[features+output]
dataset

### Scale the data in range [0,1]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_dataset = scaler.fit_transform(dataset)
scaler.fit_transform(dataset[output])
scaled_dataset = pd.DataFrame(scaled_dataset, columns=[features+output], index=dataset.index)
scaled_dataset

In [ ]:
scaled_dataset.head(12)

### Build sequence dataset

In [ ]:
X,y = df_to_X_y(scaled_dataset,features, output, n_features, window_size, step_ahead)
print('Generated {} different sequence'.format(len(X)))
print('Sequence shape: {},{}'.format(X.shape,y.shape))

In [ ]:
X[1],y[1]

### Split into training, validation and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, shuffle=False)

In [ ]:
print('The training dataset has shape: {}, {}'.format(X_train.shape, y_train.shape))
print('The validation dataset has shape: {}, {}'.format(X_val.shape, y_val.shape))
print('The test dataset has shape: {}, {}'.format(X_test.shape, y_test.shape))

### LSTM development

#### Hyperparamters optimization procedure
* **MAX_TRIALS** Number of hyperparameter combinations that will be tested by the tuner
* **EXECUTION_PER_TRIAL** The number of models that should be built and fit for each trial. Different trials have different hyperparameter values. The executions within the same trial have the same hyperparameter values. The purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model.
* **MAX_EPOCHS** Maximum number of epochs to train one model
* **MINI_BATCH** Size of subset of the dataset used to take another step in the learning process.Instead of waiting for the model to compute the whole dataset, we’re able to update its parameters more frequently. This reduces the risk of getting stuck at a local minimum, since different batches will be considered at each iteration, granting a robust convergence.
* **PATIENCE** - Number of epochs with no improvement after which training will be stopped.

In [ ]:
MAX_TRIALS = 75
EXECUTION_PER_TRIAL = 2
MAX_EPOCHS = 100
MINI_BATCH_SIZE = 16
PATIENCE = 5

build_model(kt.HyperParameters())
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective=kt.Objective('val_loss', direction='min'),
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory=directory_network,
    project_name=project_name_network,
    overwrite=True)
tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train,
             epochs=MAX_EPOCHS,
             batch_size=MINI_BATCH_SIZE,
             validation_data=(X_val, y_val),
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                         patience=PATIENCE,
                                                         verbose=1,
                                                         restore_best_weights = True)])
print('The hyperparameter search is complete. The optimal hyperparameters are:')
tuner.get_best_hyperparameters()[0].values

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir lstm-1StepAhead/lstm-1day-input

#### Retraining the model with the best hyperparameters

In [ ]:
X_train = np.vstack([X_train, X_val])
y_train = np.vstack([y_train, y_val])
X_train.shape, y_train.shape

In [ ]:
best_model = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
training_history = best_model.fit(X_train, y_train,
                                  epochs=MAX_EPOCHS,
                                  batch_size=MINI_BATCH_SIZE,
                                  )
best_model.summary()

In [ ]:
y_train_pred = best_model.predict(X_train)

In [ ]:
y_train = scaler.inverse_transform(np.reshape(y_train,(y_train.shape[0], step_ahead)))
y_train_pred = scaler.inverse_transform(y_train_pred)

In [ ]:
y_train_df = pd.DataFrame(y_train, columns=['1_day', '2_day', '3_day', '4_day','5_day','6_day','7_day'])
y_train_df.head()

In [ ]:
y_train_pred_df = pd.DataFrame(y_train_pred, columns=['1_day_pred', '2_day_pred','3_day_pred', '4_day_pred','5_day_pred', '6_day_pred', '7_day_pred'])
y_train_pred_df.head()

In [ ]:
plot_perfect_fit(y_train_df['1_day'], y_train_pred_df['1_day_pred'], str(window_size) + ' previous input time steps', 30, True, 30)

In [ ]:
plot_timeseries(y_train_df['1_day'], y_train_pred_df['1_day_pred'], str(window_size) + ' previous input time steps', 30)

In [ ]:
print('Train metrics')
compute_metrics(y_train_df['1_day'], y_train_pred_df['1_day_pred'])

#### Test the model

In [ ]:
y_test_pred = best_model.predict(X_test)

In [ ]:
y_test = scaler.inverse_transform(np.reshape(y_test,(y_test.shape[0], step_ahead)))
y_test_pred = scaler.inverse_transform(y_test_pred)

In [ ]:
y_test_df = pd.DataFrame(y_test, columns=['1_day', '2_day', '3_day', '4_day','5_day','6_day','7_day'])
y_test_df.head()

In [ ]:
y_test_pred_df = pd.DataFrame(y_test_pred, columns=['1_day_pred', '2_day_pred','3_day_pred', '4_day_pred','5_day_pred', '6_day_pred', '7_day_pred'])
y_test_pred_df.head()

In [ ]:
plot_perfect_fit(y_test_df['1_day'], y_test_pred_df['1_day_pred'], str(window_size) + ' previous input time steps', 30, True, 30)

In [ ]:
plot_timeseries(y_test_df['1_day'], y_test_pred_df['1_day_pred'], str(window_size) + ' previous input time steps', 30)

In [ ]:
print('Test metrics')
compute_metrics(y_test_df['7_day'], y_test_pred_df['7_day_pred'])

#### Save model and result

In [ ]:
data_train_pred=pd.concat([y_train_df,y_train_pred_df], axis=1)
data_train_pred.head()

In [ ]:
data_test_pred=pd.concat([y_test_df,y_test_pred_df], axis=1)
data_test_pred.head()

In [ ]:
# create a excel writer object
with pd.ExcelWriter(path_filename_results) as writer:
  data_train_pred.to_excel(writer, sheet_name='Train')
  data_test_pred.to_excel(writer, sheet_name='Test')

best_model.save(path_filename_trained_network)